In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator

from snotel_vs_snobal import (
    Variable, PlotData, start_cluster,
    basin_stations, get_station_data,
    get_isnobal_data, swe_from_zarr, UA_SWE, SNODAS_SWE,
    get_cu_boulder_swe, get_aso_swe
)

In [ ]:
basin = "Colkrem"
station_name = "Fool Creek"
add_legend = True

In [ ]:
# basin = "ERW"
# station_name = "Ivanhoe"
# add_legend = False

In [ ]:
station = basin_stations.filter(
    basin_stations.station_name == station_name
).execute()
station = station.set_crs("EPSG:4326")

In [ ]:
station.loc[0]

In [ ]:
api_data = get_station_data(station.loc[0], Variable.SWE)

In [ ]:
plot_data = {}
cluster = start_cluster(n_workers=24, memory_limit="8GB", local=False)

In [ ]:
plot_data[PlotData.STATIONS] = api_data
plot_data[PlotData.ISNOBAL] = get_isnobal_data(
    "COLKREM", station, Variable.SWE
)
plot_data[PlotData.UA] = swe_from_zarr(UA_SWE, station)
plot_data[PlotData.SNODAS] = swe_from_zarr(SNODAS_SWE, station)

In [ ]:
cluster.shutdown()

In [ ]:
plot_data[PlotData.CUB] = get_cu_boulder_swe(basin)
plot_data[PlotData.ASO] = get_aso_swe(basin)

In [ ]:
plt.figure(dpi=300, figsize=(12, 5))
ax1 = plt.gca()

line_opts = dict(alpha=0.7, zorder=1)
scatter_opts = dict(s=20, zorder=3, edgecolor="black")

x_values = plot_data[PlotData.STATIONS].index
ax1.plot(
    x_values, 
    plot_data[PlotData.STATIONS]["value"].values,
    label='SNOTEL', color="seagreen", lw=2, dashes=[6, 1], zorder=0,
)
ax1.plot(
    x_values,
    plot_data[PlotData.ISNOBAL][station_name]["specific_mass"].sel(time=x_values, method="nearest").values,
    label='iSnobal', color="goldenrod", **line_opts
)
ax1.plot(
    x_values,
    plot_data[PlotData.UA][station_name]["SWE"].sel(time=x_values).values,
    label='UA', color="royalblue", **line_opts
)
ax1.plot(
    x_values,
    plot_data[PlotData.SNODAS][station_name][Variable.SWE].sel(time=x_values).values,
    label='SNODAS', color="orchid", **line_opts
)

cub = plot_data[PlotData.CUB][
    plot_data[PlotData.CUB]["StationName"] == station_name
]
ax1.scatter(
    cub.index, cub[Variable.SWE],
    marker="^", label="CUB", color="lightsteelblue", **scatter_opts,
)
aso = plot_data[PlotData.ASO][
    plot_data[PlotData.ASO]["StationName"] == station_name
]
ax1.scatter(
    aso.index, aso[Variable.SWE],
    marker="D", label="ASO", facecolor="powderblue", **scatter_opts,
)

if add_legend:
    ax1.legend(
        loc='upper right',
        ncol=6,
        bbox_to_anchor=(1.007, 1.02),
        prop={'size': 12},
        columnspacing=0.5,
        handletextpad=0.5,
        frameon=False,
    )

ax1.set_xlim(x_values[0], x_values[-1])
ax1.set_ylim(-7, 700)
ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
ax1.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
ax1.yaxis.set_minor_locator(AutoMinorLocator())

ax1.set_ylabel("SWE [mm]")
ax1.set_xlabel("Month")
ax1.set_title(
    f"{station_name} ({station.loc[0].elevation} ft)",
    # x=0.15, y=0.93, fontweight="bold" # COLKREM
    x=0.115, y=0.93, fontweight='bold' # ERW
);